In [ ]:
pip install rdkit

In [ ]:
pip install torch

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_i

In [ ]:
pip install torch_geometric

In [ ]:
#importing the necessary modules
import csv
from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit.Chem import DataStructs
from rdkit.Chem import AllChem

import torch
from torch_geometric.data import Data

In [ ]:
import networkx as nx
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
def convert_protein_sequence_to_graph(sequence):
    graph = nx.Graph()
    # Create a node for each amino acid in the sequence.
    for amino_acid in sequence:
        graph.add_node(amino_acid)

    # Connect the nodes with edges based on the interactions between the amino acids.
    for i in range(len(sequence) - 1):
        graph.add_edge(sequence[i], sequence[i + 1])


    return graph

In [ ]:
# Load the protein sequence dataset.
protein_sequences = []
protein_functions = []
with open("trainData.csv", "r") as f:
    for line in f:
        protein_sequences.append(line.split(",")[0])
        protein_functions.append(line.split(",")[2])

In [ ]:
# Convert the protein sequences to a graph database.
graph_database = []
for protein_sequence in protein_sequences:
    graph = convert_protein_sequence_to_graph(protein_sequence)
    graph_database.append(graph)

In [ ]:
print(graph)

Graph with 20 nodes and 178 edges


In [ ]:
def encode_protein_sequence_with_one_hot_encoding(protein_sequence):
    graph_features = []
    for amino_acid in protein_sequence:
        graph_features.append(np.array([1, 0, 0, 0, 0, 0, 0, 0, 0], dtype=np.float32))

    graph_features = np.array(graph_features)
    return graph_features

In [ ]:
# Encode the protein sequences into graph features using one-hot encoding.
graph_features = []
for protein_sequence in protein_sequences:
    graph_features.append(encode_protein_sequence_with_one_hot_encoding(protein_sequence))

In [ ]:
# Save the graph features to a file.
np.save("graph_features.npy", graph_features)

/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py:521: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [ ]:
print(graph_features)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import DataLoader
from torch_geometric.nn import GINConv
from torch_geometric.datasets import TUDataset

# Define your GIN model class.
class GINModel(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super(GINModel, self).__init__()
        self.conv1 = GINConv(nn=nn.Sequential(nn.Linear(num_features, 32), nn.ReLU()))
        self.conv2 = GINConv(nn=nn.Sequential(nn.Linear(32, 64), nn.ReLU()))
        self.fc = nn.Linear(64, num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        x = torch.mean(x, dim=0)  # Global pooling (mean)
        x = self.fc(x)
        return x

In [ ]:
# Create an instance of the GraphSAGE model
model = GraphSAGE(num_features=128, num_classes=10)

In [ ]:
import csv
import networkx as nx

# Load the graph database from the CSV file.
graph_database = nx.Graph()
with open("trainData.csv", "r") as f:
    for line in f:
        # Split the line into columns.
        columns = line.split(",")

        # Create a node for each amino acid in the sequence.
        for amino_acid in columns[0]:
            graph_database.add_node(amino_acid)

        # Connect the nodes with edges based on the interactions between the amino acids.
        for i in range(len(columns[0]) - 1):
            graph_database.add_edge(columns[0][i], columns[0][i + 1])

In [ ]:
# Get the edge index.
edge_index = []
for edge in graph_database.edges():
    edge_index.append(edge)

In [ ]:
print(edge_index)

[('M', 'S'), ('M', 'Q'), ('M', 'L'), ('M', 'Y'), ('M', 'I'), ('M', 'R'), ('M', 'K'), ('M', 'M'), ('M', 'A'), ('M', 'D'), ('M', 'E'), ('M', 'P'), ('M', 'F'), ('M', 'G'), ('M', 'W'), ('M', 'V'), ('M', 'T'), ('M', 'H'), ('M', 'N'), ('M', 'C'), ('M', 'X'), ('S', 'L'), ('S', 'F'), ('S', 'I'), ('S', 'V'), ('S', 'H'), ('S', 'Y'), ('S', 'S'), ('S', 'G'), ('S', 'R'), ('S', 'W'), ('S', 'N'), ('S', 'E'), ('S', 'C'), ('S', 'T'), ('S', 'K'), ('S', 'D'), ('S', 'A'), ('S', 'Q'), ('S', 'P'), ('S', 'X'), ('S', 'U'), ('S', 'B'), ('L', 'Q'), ('L', 'I'), ('L', 'K'), ('L', 'A'), ('L', 'D'), ('L', 'E'), ('L', 'F'), ('L', 'L'), ('L', 'R'), ('L', 'H'), ('L', 'G'), ('L', 'T'), ('L', 'Y'), ('L', 'V'), ('L', 'C'), ('L', 'N'), ('L', 'P'), ('L', 'W'), ('L', 'X'), ('L', 'B'), ('L', 'U'), ('Q', 'R'), ('Q', 'F'), ('Q', 'N'), ('Q', 'I'), ('Q', 'A'), ('Q', 'G'), ('Q', 'V'), ('Q', 'K'), ('Q', 'H'), ('Q', 'C'), ('Q', 'Y'), ('Q', 'T'), ('Q', 'E'), ('Q', 'P'), ('Q', 'D'), ('Q', 'Q'), ('Q', 'W'), ('Q', 'X'), ('Q', 'Z'), ('Q

In [ ]:
edge_list =[('M', 'S'), ('M', 'Q'), ('M', 'L'), ('M', 'Y'), ('M', 'I'), ('M', 'R'), ('M', 'K'), ('M', 'M'), ('M', 'A'), ('M', 'D'), ('M', 'E'), ('M', 'P'), ('M', 'F'), ('M', 'G'), ('M', 'W'), ('M', 'V'), ('M', 'T'), ('M', 'H'), ('M', 'N'), ('M', 'C'), ('M', 'X'), ('S', 'L'), ('S', 'F'), ('S', 'I'), ('S', 'V'), ('S', 'H'), ('S', 'Y'), ('S', 'S'), ('S', 'G'), ('S', 'R'), ('S', 'W'), ('S', 'N'), ('S', 'E'), ('S', 'C'), ('S', 'T'), ('S', 'K'), ('S', 'D'), ('S', 'A'), ('S', 'Q'), ('S', 'P'), ('S', 'X'), ('S', 'U'), ('S', 'B'), ('L', 'Q'), ('L', 'I'), ('L', 'K'), ('L', 'A'), ('L', 'D'), ('L', 'E'), ('L', 'F'), ('L', 'L'), ('L', 'R'), ('L', 'H'), ('L', 'G'), ('L', 'T'), ('L', 'Y'), ('L', 'V'), ('L', 'C'), ('L', 'N'), ('L', 'P'), ('L', 'W'), ('L', 'X'), ('L', 'B'), ('L', 'U'), ('Q', 'R'), ('Q', 'F'), ('Q', 'N'), ('Q', 'I'), ('Q', 'A'), ('Q', 'G'), ('Q', 'V'), ('Q', 'K'), ('Q', 'H'), ('Q', 'C'), ('Q', 'Y'), ('Q', 'T'), ('Q', 'E'), ('Q', 'P'), ('Q', 'D'), ('Q', 'Q'), ('Q', 'W'), ('Q', 'X'), ('Q', 'Z'), ('Q', 'U'), ('Q', 'B'), ('I', 'K'), ('I', 'Y'), ('I', 'C'), ('I', 'P'), ('I', 'R'), ('I', 'F'), ('I', 'A'), ('I', 'E'), ('I', 'G'), ('I', 'H'), ('I', 'N'), ('I', 'T'), ('I', 'D'), ('I', 'I'), ('I', 'V'), ('I', 'W'), ('I', 'U'), ('I', 'X'), ('I', 'Z'), ('K', 'P'), ('K', 'E'), ('K', 'F'), ('K', 'K'), ('K', 'C'), ('K', 'A'), ('K', 'D'), ('K', 'V'), ('K', 'R'), ('K', 'T'), ('K', 'G'), ('K', 'N'), ('K', 'H'), ('K', 'W'), ('K', 'Y'), ('K', 'X'), ('K', 'B'), ('P', 'E'), ('P', 'F'), ('P', 'R'), ('P', 'N'), ('P', 'D'), ('P', 'V'), ('P', 'P'), ('P', 'G'), ('P', 'T'), ('P', 'Y'), ('P', 'W'), ('P', 'C'), ('P', 'A'), ('P', 'H'), ('P', 'X'), ('P', 'U'), ('P', 'Z'), ('E', 'F'), ('E', 'E'), ('E', 'A'), ('E', 'R'), ('E', 'V'), ('E', 'T'), ('E', 'H'), ('E', 'D'), ('E', 'G'), ('E', 'N'), ('E', 'W'), ('E', 'Y'), ('E', 'C'), ('E', 'X'), ('E', 'Z'), ('A', 'F'), ('A', 'R'), ('A', 'V'), ('A', 'A'), ('A', 'C'), ('A', 'Y'), ('A', 'G'), ('A', 'D'), ('A', 'W'), ('A', 'T'), ('A', 'H'), ('A', 'N'), ('A', 'X'), ('F', 'D'), ('F', 'N'), ('F', 'T'), ('F', 'V'), ('F', 'R'), ('F', 'G'), ('F', 'F'), ('F', 'Y'), ('F', 'H'), ('F', 'W'), ('F', 'C'), ('F', 'U'), ('F', 'X'), ('Y', 'R'), ('Y', 'N'), ('Y', 'H'), ('Y', 'T'), ('Y', 'G'), ('Y', 'V'), ('Y', 'D'), ('Y', 'C'), ('Y', 'W'), ('Y', 'Y'), ('Y', 'X'), ('Y', 'U'), ('Y', 'B'), ('R', 'R'), ('R', 'G'), ('R', 'D'), ('R', 'T'), ('R', 'V'), ('R', 'C'), ('R', 'N'), ('R', 'H'), ('R', 'W'), ('R', 'X'), ('R', 'U'), ('G', 'D'), ('G', 'V'), ('G', 'C'), ('G', 'T'), ('G', 'N'), ('G', 'H'), ('G', 'G'), ('G', 'W'), ('G', 'X'), ('G', 'U'), ('G', 'B'), ('D', 'N'), ('D', 'H'), ('D', 'V'), ('D', 'T'), ('D', 'W'), ('D', 'C'), ('D', 'D'), ('D', 'X'), ('C', 'V'), ('C', 'H'), ('C', 'T'), ('C', 'C'), ('C', 'N'), ('C', 'W'), ('C', 'X'), ('C', 'U'), ('V', 'N'), ('V', 'T'), ('V', 'H'), ('V', 'V'), ('V', 'W'), ('V', 'X'), ('V', 'U'), ('N', 'H'), ('N', 'T'), ('N', 'W'), ('N', 'N'), ('N', 'X'), ('H', 'T'), ('H', 'W'), ('H', 'H'), ('H', 'U'), ('T', 'W'), ('T', 'T'), ('T', 'X'), ('T', 'U'), ('W', 'W'), ('W', 'X'), ('X', 'X'), ('B', 'B')]

In [ ]:
# Create a dictionary to map node labels to unique integer IDs
node_to_id = {node: i for i, node in enumerate(set(node for edge in edge_list for node in edge))}

# Convert the edge list to an integer tensor
edge_index = torch.tensor([[node_to_id[edge[0]], node_to_id[edge[1]]] for edge in edge_list], dtype=torch.long).t()

In [ ]:
import networkx as nx

# Assuming you have created the `graph_database` as described in your code.

# Define the filename for the GraphML file.
file_name = "graph_database.graphml"

# Save the graph to a GraphML file.
nx.write_graphml(graph_database, file_name)

print(f"Graph database saved to {file_name}")

Graph database saved to graph_database.graphml


In [104]:
import torch
import networkx as nx

# Example edge_list containing string labels for nodes.
edge_list = [('M', 'S'), ('M', 'Q'), ('M', 'L'), ('M', 'Y'), ('M', 'I'), ('M', 'R'), ('M', 'K'), ('M', 'M'), ('M', 'A'), ('M', 'D'), ('M', 'E'), ('M', 'P'), ('M', 'F'), ('M', 'G'), ('M', 'W'), ('M', 'V'), ('M', 'T'), ('M', 'H'), ('M', 'N'), ('M', 'C'), ('M', 'X'), ('S', 'L'), ('S', 'F'), ('S', 'I'), ('S', 'V'), ('S', 'H'), ('S', 'Y'), ('S', 'S'), ('S', 'G'), ('S', 'R'), ('S', 'W'), ('S', 'N'), ('S', 'E'), ('S', 'C'), ('S', 'T'), ('S', 'K'), ('S', 'D'), ('S', 'A'), ('S', 'Q'), ('S', 'P'), ('S', 'X'), ('S', 'U'), ('S', 'B'), ('L', 'Q'), ('L', 'I'), ('L', 'K'), ('L', 'A'), ('L', 'D'), ('L', 'E'), ('L', 'F'), ('L', 'L'), ('L', 'R'), ('L', 'H'), ('L', 'G'), ('L', 'T'), ('L', 'Y'), ('L', 'V'), ('L', 'C'), ('L', 'N'), ('L', 'P'), ('L', 'W'), ('L', 'X'), ('L', 'B'), ('L', 'U'), ('Q', 'R'), ('Q', 'F'), ('Q', 'N'), ('Q', 'I'), ('Q', 'A'), ('Q', 'G'), ('Q', 'V'), ('Q', 'K'), ('Q', 'H'), ('Q', 'C'), ('Q', 'Y'), ('Q', 'T'), ('Q', 'E'), ('Q', 'P'), ('Q', 'D'), ('Q', 'Q'), ('Q', 'W'), ('Q', 'X'), ('Q', 'Z'), ('Q', 'U'), ('Q', 'B'), ('I', 'K'), ('I', 'Y'), ('I', 'C'), ('I', 'P'), ('I', 'R'), ('I', 'F'), ('I', 'A'), ('I', 'E'), ('I', 'G'), ('I', 'H'), ('I', 'N'), ('I', 'T'), ('I', 'D'), ('I', 'I'), ('I', 'V'), ('I', 'W'), ('I', 'U'), ('I', 'X'), ('I', 'Z'), ('K', 'P'), ('K', 'E'), ('K', 'F'), ('K', 'K'), ('K', 'C'), ('K', 'A'), ('K', 'D'), ('K', 'V'), ('K', 'R'), ('K', 'T'), ('K', 'G'), ('K', 'N'), ('K', 'H'), ('K', 'W'), ('K', 'Y'), ('K', 'X'), ('K', 'B'), ('P', 'E'), ('P', 'F'), ('P', 'R'), ('P', 'N'), ('P', 'D'), ('P', 'V'), ('P', 'P'), ('P', 'G'), ('P', 'T'), ('P', 'Y'), ('P', 'W'), ('P', 'C'), ('P', 'A'), ('P', 'H'), ('P', 'X'), ('P', 'U'), ('P', 'Z'), ('E', 'F'), ('E', 'E'), ('E', 'A'), ('E', 'R'), ('E', 'V'), ('E', 'T'), ('E', 'H'), ('E', 'D'), ('E', 'G'), ('E', 'N'), ('E', 'W'), ('E', 'Y'), ('E', 'C'), ('E', 'X'), ('E', 'Z'), ('A', 'F'), ('A', 'R'), ('A', 'V'), ('A', 'A'), ('A', 'C'), ('A', 'Y'), ('A', 'G'), ('A', 'D'), ('A', 'W'), ('A', 'T'), ('A', 'H'), ('A', 'N'), ('A', 'X'), ('F', 'D'), ('F', 'N'), ('F', 'T'), ('F', 'V'), ('F', 'R'), ('F', 'G'), ('F', 'F'), ('F', 'Y'), ('F', 'H'), ('F', 'W'), ('F', 'C'), ('F', 'U'), ('F', 'X'), ('Y', 'R'), ('Y', 'N'), ('Y', 'H'), ('Y', 'T'), ('Y', 'G'), ('Y', 'V'), ('Y', 'D'), ('Y', 'C'), ('Y', 'W'), ('Y', 'Y'), ('Y', 'X'), ('Y', 'U'), ('Y', 'B'), ('R', 'R'), ('R', 'G'), ('R', 'D'), ('R', 'T'), ('R', 'V'), ('R', 'C'), ('R', 'N'), ('R', 'H'), ('R', 'W'), ('R', 'X'), ('R', 'U'), ('G', 'D'), ('G', 'V'), ('G', 'C'), ('G', 'T'), ('G', 'N'), ('G', 'H'), ('G', 'G'), ('G', 'W'), ('G', 'X'), ('G', 'U'), ('G', 'B'), ('D', 'N'), ('D', 'H'), ('D', 'V'), ('D', 'T'), ('D', 'W'), ('D', 'C'), ('D', 'D'), ('D', 'X'), ('C', 'V'), ('C', 'H'), ('C', 'T'), ('C', 'C'), ('C', 'N'), ('C', 'W'), ('C', 'X'), ('C', 'U'), ('V', 'N'), ('V', 'T'), ('V', 'H'), ('V', 'V'), ('V', 'W'), ('V', 'X'), ('V', 'U'), ('N', 'H'), ('N', 'T'), ('N', 'W'), ('N', 'N'), ('N', 'X'), ('H', 'T'), ('H', 'W'), ('H', 'H'), ('H', 'U'), ('T', 'W'), ('T', 'T'), ('T', 'X'), ('T', 'U'), ('W', 'W'), ('W', 'X'), ('X', 'X'), ('B', 'B')]

# Create a dictionary to map node labels to unique integer IDs.
node_to_id = {node: i for i, node in enumerate(set(node for edge in edge_list for node in edge))}

# Convert the edge_list to an integer tensor.
edge_index = torch.tensor([[node_to_id[edge[0]], node_to_id[edge[1]]] for edge in edge_list], dtype=torch.long).t()

print(edge_index)

tensor([[ 3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
          3,  3,  3, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22,
         22, 22, 22, 22, 22, 22, 22,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
          4,  4,  4,  4,  4,  4,  4,  4,  4,  4, 13, 13, 13, 13, 13, 13, 13, 13,
         13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 10, 10, 10, 10, 10,
         10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 19, 19, 19, 19,
         19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,  8,  8,  8,  8,  8,
          8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  5,  5,  5,  5,  5,  5,
          5,  5,  5,  5,  5,  5,  5,  5,  5,  9,  9,  9,  9,  9,  9,  9,  9,  9,
          9,  9,  9,  9, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 18,
         18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2, 20, 20,
         20, 20, 20, 20, 20,

In [113]:
print(num_classes)

128


In [105]:
# Create a PyTorch Geometric Data object.
data = Data(x=graph_features, edge_index=edge_index)

In [106]:
# Define your data loader.
data_loader = DataLoader(data, batch_size=64, shuffle=True)

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [114]:
# Initialize your GIN model, loss function, and optimizer.
num_features = len(graph_features[0])  # Use the length of the first feature vector

model = GINModel(num_features=num_features, num_classes=num_classes)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [119]:
import networkx as nx

# Specify the path to your GraphML file.
graphml_file = "graph_database.graphml"

# Load the graph from the GraphML file.
graph_database = nx.read_graphml(graphml_file)

In [122]:
# Create a data loader
data_loader = DataLoader(graph_database, batch_size=32)